In [3]:
import numpy as np, pandas as pd, os
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor as RFReg

wrk='C:\\Users\\Lenovo\\Documents\\GeekBrains\\Python DataScience Kurs'
train=pd.read_csv(os.path.join(wrk,'train.csv'))

train=train.drop(['Id'],axis=1) #удаляем стоблбец id - он нам не нужен
cols=train.columns.to_numpy()

res={}

for x in cols:
    res[x]=train[x].value_counts().sort_index()

In [4]:
print(res)

{'DistrictId': 0       48
1      652
2       78
3       82
4       11
      ... 
202      2
205      1
207      1
208      2
209      1
Name: DistrictId, Length: 205, dtype: int64, 'Rooms': 0.0        8
1.0     3705
2.0     3880
3.0     2235
4.0      150
5.0       18
6.0        1
10.0       2
19.0       1
Name: Rooms, dtype: int64, 'Square': 1.136859      1
1.988943      1
2.377248      1
2.596351      1
2.954309      1
             ..
212.932361    1
275.645284    1
409.425181    1
604.705972    1
641.065193    1
Name: Square, Length: 10000, dtype: int64, 'LifeSquare': 0.370619       1
0.641822       1
0.795539       1
0.873147       1
1.049867       1
              ..
263.542020     1
410.639749     1
461.463614     1
638.163193     1
7480.592129    1
Name: LifeSquare, Length: 7887, dtype: int64, 'KitchenSquare': 0.0        697
1.0       2460
2.0          4
3.0         22
4.0         39
5.0       1169
6.0       1038
7.0        609
8.0       1306
9.0        843
10.0      1075
11.0    

In [ ]:
'''
Результаты анализа значений:
    
    DistrictId - 205 возможных значений. Прямой/обратной зависимости (чем больше id, тем лучше/хуже район)
            стоимости квартиры от данного параметра не предполагается. Поэтому отнесем к дискретным;
    Rooms - 9 различных значений. Предполагается зависимость цены квартиры от кол-ва комнат, поэтому отнесем этот
            параметр к непрерывным;
    Square - 10000 различных значений. ОТносим к непрерывным признакам;
    LifeSquare - 7887 различных значений. Относим к непрерывным значениям; Значения nan будем заменять значениями
            Square-KitchenSquare ПОСЛЕ редактирования KitchenSquare;
    KitchenSquare - 58 различных значений. Предполагается зависимость цены квартиры от площади кухни, поэтому отнесем этот
            параметр к непрерывным. Почти 25% значений равны 1 => заменим их на медиану без учета 1.
            Есть ошибочные значения 1970 и 2014 (видимо, год постройки дома);
    Floor - 33 различных значения. Предполагается зависимость цены квартиры от ее этажа, поэтому отнесем этот
            параметр к непрерывным;
    HouseFloor - 44 различных значения. Предполагается зависимость цены квартиры от этажности дома, поэтому
            отнесем этот параметр к непрерывным; ОБРАБОТКА ОШИБОЧНЫХ ЗНАЧЕНИЙ: Floor <= HouseFloor и HouseFloor!=0;
    HouseYear - 97 различных значений. Предполагается зависимость цены квартиры от года постройки дома, поэтому
            отнесем этот параметр к непрерывным; Ошибочные значения: 4968 и 20052011;
    Ecology_1 - 129 непрерывных значений. Отнесем этот параметр к непрерывным;
    Ecology_2 - 2 значения (A/B). Отнесем этот параметр к дискретным;
    Ecology_3 - 2 значения (A/B). Отнесем этот параметр к дискретным;
    Social_1 - 51 различное значение. Предполагается зависимость цены квартиры от социальных условий, поэтому
            отнесем этот параметр к непрерывным;
    Social_2 - 142 различных значения. Аналогично;
    Social_3 - 30 различных значений. Аналогично;
    Healthcare_1 - 79 различных значений. Аналогично;
    Helthcare_2 - 7 различных значений. Из-за этого отнесем этот параметр к дискретным;
    Shops_1 - 16 различных значений. Предполагается зависимость цены квартиры от доступности магазинов, поэтому
            отнесем этот параметр к непрерывным;
    Shops_2 - 2 значения (A/B). Отнесем этот параметр к дискретным;
    
'''

In [5]:
#обработка обучающей выборки
#редактирование значений площади кухни
kitch_sq=train['KitchenSquare']
kitch_sq_med=kitch_sq.loc[kitch_sq!=1].median() #медианное значение площади кухни
kitch_sq[kitch_sq==1]=kitch_sq_med #заменяем медиану

#редактируем значения nan жилой площади
li_sq_nan=train.loc[train['LifeSquare'].apply(np.isnan)]
sq_nan=li_sq_nan['Square']-li_sq_nan['KitchenSquare']
sq=train['LifeSquare']
sq2=pd.concat([sq,sq_nan],axis=1) #соединяем для подгонки по индексу. Получился dataframe с 2 столбцами

for i in range(len(sq2)):
    if np.isnan(sq2['LifeSquare'][i]):
        sq2['LifeSquare'][i]=sq2[0][i]
        
sq2=sq2.drop(0,axis=1) 
train['LifeSquare']=sq2


<ipython-input-5-8fff003d164a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kitch_sq[kitch_sq==1]=kitch_sq_med #заменяем медиану


In [6]:
#редактируем значения первого столбца здравоохранения
h1=train['Healthcare_1'].median() #900
train['Healthcare_1']=train['Healthcare_1'].fillna(h1)

#редактируем ошибочные значения
print(train.loc[train['KitchenSquare']>=1970])
train['KitchenSquare'][1064]=0
train['KitchenSquare'][8584]=8

print(train['HouseYear'].loc[train['HouseYear']>2021])
train['HouseYear'][1497]=2011
train['HouseYear'][4189]=1968

#анализ данных об этажах
hou_err1=train.loc[(train['Floor']>train['HouseFloor'])]
hou_err2=train.loc[(train['Floor']==0)] #пусто
hou_err3=train.loc[(train['HouseFloor']==0)]
print(len(hou_err1),len(hou_err2),len(hou_err3))
print(len(hou_err1)+len(hou_err2)+len(hou_err3))


      DistrictId  Rooms     Square  LifeSquare  KitchenSquare  Floor  \
1064          62    1.0  47.100719   46.447960         2014.0      4   
8584          81    1.0  32.276663   19.278394         1970.0      6   

      HouseFloor  HouseYear  Ecology_1 Ecology_2 Ecology_3  Social_1  \
1064         1.0       2014   0.072158         B         B         2   
8584         1.0       1977   0.006076         B         B        30   

      Social_2  Social_3  Healthcare_1  Helthcare_2  Shops_1 Shops_2  \
1064       629         1         900.0            0        0       A   
8584      5285         0         645.0            6        6       B   

              Price  
1064  108337.484207  
8584  105539.556275  
1497    20052011
4189        4968
Name: HouseYear, dtype: int64
1825 0 269
2094


<ipython-input-6-339ced2e64b1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['KitchenSquare'][1064]=0
<ipython-input-6-339ced2e64b1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['KitchenSquare'][8584]=8
<ipython-input-6-339ced2e64b1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['HouseYear'][1497]=2011
<ipython-input-6-339ced2e64b1>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [7]:
'''
Более 20% квартир имеют неверные данные об этажах.
    Как поступить:
    Если этаж квартиры больше этажности дома, то сделаем этажность дома равной этажу дома
    Если этажность дома = 0, то сделаем то же самое
'''
hou_err1['HouseFloor']=hou_err1['Floor']
hou_err3['HouseFloor']=hou_err3['Floor']

hou_err1=hou_err1['HouseFloor']
hou_err3=hou_err3['HouseFloor']

#редактируем исходный столбец
train_hf=train['HouseFloor']
train_hf=pd.concat([train_hf,hou_err1,hou_err3],axis=1)
train_hf.columns=['HouseFloor',1,2]

#заменяем значения
for i in range(len(train_hf)):
    if not np.isnan(train_hf[1][i]):
        train_hf['HouseFloor'][i]=train_hf[1][i]
    elif not np.isnan(train_hf[2][i]):
        train_hf['HouseFloor'][i]=train_hf[2][i]

train_hf=train_hf.drop([1,2],axis=1)
train['HouseFloor']=train_hf

<ipython-input-7-be0d8dc6a4a7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hou_err1['HouseFloor']=hou_err1['Floor']
<ipython-input-7-be0d8dc6a4a7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hou_err3['HouseFloor']=hou_err3['Floor']


In [8]:
#анализируем стоимость квартиры от параметров Ecology_2, Ecology_3, Shops_2
train_symb_p=train[['Ecology_2','Ecology_3','Shops_2','Price']]
train_symb_agg_ec2=train_symb_p.groupby(by=['Ecology_2']).agg({'Price': ['min','max','mean','median']})
train_symb_agg_ec3=train_symb_p.groupby(by=['Ecology_3']).agg({'Price': ['min','max','mean','median']})
train_symb_agg_sh2=train_symb_p.groupby(by=['Shops_2']).agg({'Price': ['min','max','mean','median']})
print('Зависимость от Ecology_2: ',train_symb_agg_ec2,end='\n\n')
print('Зависимость от Ecology_3: ',train_symb_agg_ec3,end='\n\n')
print('Зависимость от Shops_2: ',train_symb_agg_sh2,end='\n\n')

'''
Средняя и меданная стоимость квартиры при параметрах Ecology_2 и Shops_2 выше при значении параметра B,
а при параметре Ecology_3 - при значении параметра A. Поэтому большей цене присваиваем соответствующим параметрам
значение 1, а другому значению - 0:
'''

train.loc[train['Ecology_2']=='B','Ecology_2']=1
train.loc[train['Ecology_2']=='A','Ecology_2']=0
train.loc[train['Ecology_3']=='A','Ecology_3']=1
train.loc[train['Ecology_3']=='B','Ecology_3']=0
train.loc[train['Shops_2']=='B','Shops_2']=1
train.loc[train['Shops_2']=='A','Shops_2']=0

Зависимость от Ecology_2:                     Price                                             
                     min            max           mean         median
Ecology_2                                                            
A          108232.647619  480940.075839  193139.717006  182516.558181
B           59174.778028  633233.466570  214344.544223  192371.633485

Зависимость от Ecology_3:                    Price                                             
                    min            max           mean         median
Ecology_3                                                           
A          84577.918669  624549.354546  243179.560704  211889.942902
B          59174.778028  633233.466570  213317.654992  191668.400065

Зависимость от Shops_2:                  Price                                             
                  min            max           mean         median
Shops_2                                                           
A        67343.133878  

In [9]:
'''
Обработка данных завершена. Начинаем анализ значимости признаков и обучения модели
'''

feats_tab=train.copy()
answ_tab=feats_tab['Price']
feats_tab=feats_tab.drop(['Price'],axis=1)

feats_tab=feats_tab.to_numpy()
answ_tab=answ_tab.to_numpy()

reg=DecisionTreeRegressor(criterion='mae')
reg.fit(feats_tab,answ_tab)
feat_importance=reg.tree_.compute_feature_importances(normalize=True)
feat_importance=np.vstack((cols[:18],feat_importance))
feat_importance=np.transpose(feat_importance)
feat_importance=pd.DataFrame(feat_importance)
feat_importance.columns=['Param','Importance']
feat_importance=feat_importance.sort_values(by='Importance',ascending=False)
feat_importance=feat_importance.reset_index().drop(['index'],axis=1) #сброс индекса
print(feat_importance)

print('Сумма значимости первых 5 признаков: ',str(sum(feat_importance['Importance'][:5])))
print('Сумма значимости первых 10 признаков: ',str(sum(feat_importance['Importance'][:10])))

feats_top10=feat_importance.loc[:9]
print('10 наиболее значимых признаков:\n',feats_top10)

            Param   Importance
0          Square     0.196673
1           Rooms      0.18727
2        Social_2    0.0774482
3        Social_1    0.0763972
4      LifeSquare    0.0747941
5      DistrictId    0.0594486
6       Ecology_1    0.0590498
7       HouseYear     0.057061
8           Floor    0.0484812
9        Social_3     0.041825
10     HouseFloor    0.0398981
11  KitchenSquare    0.0244965
12   Healthcare_1    0.0237713
13        Shops_1    0.0202471
14    Helthcare_2    0.0105854
15      Ecology_3   0.00112647
16        Shops_2   0.00107891
17      Ecology_2  0.000348821
Сумма значимости первых 5 признаков:  0.6125820070672847
Сумма значимости первых 10 признаков:  0.8784474799137746
10 наиболее значимых признаков:
         Param Importance
0      Square   0.196673
1       Rooms    0.18727
2    Social_2  0.0774482
3    Social_1  0.0763972
4  LifeSquare  0.0747941
5  DistrictId  0.0594486
6   Ecology_1  0.0590498
7   HouseYear   0.057061
8       Floor  0.0484812
9    Social_3

In [10]:
'''
Таким образом, в первую очередь стоимость квартиры зависит от площади и кол-ва комнат, в меньшей степени
влияют социальные условия, жилая площадь, район, экология (Ecology_1), год постройки дома и этаж квартиры.

В наименьшей степени на стоимость квартиры влияют площадь кухни, близость к магазинам, торговым центрам,
учреждениям здравоохранения, а также экологические категории под номерами 2 и 3;
'''

#подготовка тестовой выборки к разделению для обучения множества регрессоров
X=train.copy()
y=X['Price']
X=X.drop(['Price'],axis=1)
X=X.to_numpy()
y=y.to_numpy()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)

#подготовка модели и настройка алгоритма перебора
reg=DecisionTreeRegressor(criterion='mae')
params={'max_depth': np.arange(5,15)} #сначала перебор по глубине дерева
GS=GridSearchCV(reg, params)
GS.fit(X_train,y_train)

test_react=GS.best_estimator_.predict(X_test)
r2_1=r2_score(y_test,test_react)
print(r2_1,GS.best_params_)

0.6044904625197393 {'max_depth': 8}


In [11]:
'''Наибольшее значение r2 показало дерево, имеющее максимальную глубину 8. Оно равно 0.6048.
   Поэтому проведем еще один поиск, на этот раз по min_samples_leaf. Макс. глубину дерева
   зафиксируем на 8'''
reg2=DecisionTreeRegressor(criterion='mae',max_depth=8)
params2={'min_samples_leaf': np.arange(2,10)} #затем перебор по мнинмальному кол-ву объектов в узле
GS2=GridSearchCV(reg2, params2)
GS2.fit(X_train,y_train)

test_react=GS2.best_estimator_.predict(X_test)
r2_2=r2_score(y_test,test_react)
print(r2_2,GS2.best_params_)

0.6263059502161779 {'min_samples_leaf': 2}


In [12]:
'''Наибольшее значение r2 показало дерево, имеющее минимальное кол-во
   объектов в узле, равное 2, при мин. глубине дерева в 8. Оно немного улучшилось и стало
   равно 0.6288.
   
   Создадим ансамбли из деревьев с подобными параметрами и обучим его. Кол-ва деревьев выберем
   равными 5, 10, 20, 30, 50, 100, 200, 500, 1000.
   
   ОБучим модель с полученными параметрами уже на всей обучающей выборке'''

ens1=RFReg(n_estimators=5,max_depth=8,min_samples_leaf=2)
ens1.fit(X_train,y_train)
test_react=ens1.predict(X_test)
r2_ens1=r2_score(y_test,test_react)

ens2=RFReg(n_estimators=10,max_depth=8,min_samples_leaf=2)
ens2.fit(X_train,y_train)
test_react=ens2.predict(X_test)
r2_ens2=r2_score(y_test,test_react)

ens3=RFReg(n_estimators=20,max_depth=8,min_samples_leaf=2)
ens3.fit(X_train,y_train)
test_react=ens3.predict(X_test)
r2_ens3=r2_score(y_test,test_react)

ens4=RFReg(n_estimators=30,max_depth=8,min_samples_leaf=2)
ens4.fit(X_train,y_train)
test_react=ens4.predict(X_test)
r2_ens4=r2_score(y_test,test_react)

ens5=RFReg(n_estimators=50,max_depth=8,min_samples_leaf=2)
ens5.fit(X_train,y_train)
test_react=ens5.predict(X_test)
r2_ens5=r2_score(y_test,test_react)

ens6=RFReg(n_estimators=100,max_depth=8,min_samples_leaf=2)
ens6.fit(X_train,y_train)
test_react=ens6.predict(X_test)
r2_ens6=r2_score(y_test,test_react)

ens7=RFReg(n_estimators=200,max_depth=8,min_samples_leaf=2)
ens7.fit(X_train,y_train)
test_react=ens7.predict(X_test)
r2_ens7=r2_score(y_test,test_react)

ens8=RFReg(n_estimators=500,max_depth=8,min_samples_leaf=2)
ens8.fit(X_train,y_train)
test_react=ens8.predict(X_test)
r2_ens8=r2_score(y_test,test_react)

ens9=RFReg(n_estimators=1000,max_depth=8,min_samples_leaf=2)
ens9.fit(X_train,y_train)
test_react=ens9.predict(X_test)
r2_ens9=r2_score(y_test,test_react)

print('5 - {:.4f}\n10 - {:.4f}\n20 - {:.4f}\n30 - {:.4f}\n50 - {:.4f}\n100 - {:.4f}\n200 - {:.4f}\n500 - {:.4f}\n1000 - {:.4f}\n'\
      .format(r2_ens1,r2_ens2,r2_ens3,r2_ens4,r2_ens5,r2_ens6,r2_ens7,r2_ens8,r2_ens9))

5 - 0.6597
10 - 0.6761
20 - 0.6863
30 - 0.6923
50 - 0.6870
100 - 0.6885
200 - 0.6900
500 - 0.6907
1000 - 0.6897



In [16]:
'''
Наилучший результат показал ансамбль из 30 деревьев (r2=0.6923). Обучим его на полной выборке:
'''
reg=RFReg(n_estimators=30,max_depth=8,min_samples_leaf=2)
reg.fit(feats_tab,answ_tab)

RandomForestRegressor(max_depth=8, min_samples_leaf=2, n_estimators=30)

In [17]:
#проверка модели
test=pd.read_csv(os.path.join(wrk,'test.csv'))
test=test.drop(['Id'],axis=1) #удаляем стоблбец id - он нам не нужен

'''В тестовых данных также имеются пропуски и неточности. Обработаем их точно так же, как и обучающие данные:'''

#редактирование значений площади кухни
kitch_sq=test['KitchenSquare']
kitch_sq_med=kitch_sq.loc[kitch_sq!=1].median() #медианное значение площади кухни
kitch_sq[kitch_sq==1]=kitch_sq_med #заменяем медиану

#редактируем значения nan жилой площади
li_sq_nan=test.loc[test['LifeSquare'].apply(np.isnan)]
sq_nan=li_sq_nan['Square']-li_sq_nan['KitchenSquare']
sq=test['LifeSquare']
sq2=pd.concat([sq,sq_nan],axis=1) #соединяем для подгонки по индексу. Получился dataframe с 2 столбцами

for i in range(len(sq2)):
    if np.isnan(sq2['LifeSquare'][i]):
        sq2['LifeSquare'][i]=sq2[0][i]
        
sq2=sq2.drop(0,axis=1) 
test['LifeSquare']=sq2

#редактируем значения первого столбца здравоохранения
h1=test['Healthcare_1'].median() #900
test['Healthcare_1']=test['Healthcare_1'].fillna(h1)

#проверяем ошибочные значения
print(test.loc[test['KitchenSquare']>=1970])
print(test['HouseYear'].loc[test['HouseYear']>2021])

#анализ данных об этажах
hou_err1=test.loc[(test['Floor']>test['HouseFloor'])]
hou_err2=test.loc[(test['Floor']==0)] #пусто
hou_err3=test.loc[(test['HouseFloor']==0)]
print(len(hou_err1),len(hou_err2),len(hou_err3))
print(len(hou_err1)+len(hou_err2)+len(hou_err3))

'''
В тестовой выборке также более 20% квартир имеют неверные данные об этажах.
    Как поступить:
    Если этаж квартиры больше этажности дома, то сделаем этажность дома равной этажу дома
    Если этажность дома = 0, то сделаем то же самое
'''
hou_err1['HouseFloor']=hou_err1['Floor']
hou_err3['HouseFloor']=hou_err3['Floor']

hou_err1=hou_err1['HouseFloor']
hou_err3=hou_err3['HouseFloor']

#редактируем исходный столбец
test_hf=test['HouseFloor']
test_hf=pd.concat([test_hf,hou_err1,hou_err3],axis=1)
test_hf.columns=['HouseFloor',1,2]

#заменяем значения
for i in range(len(test_hf)):
    if not np.isnan(test_hf[1][i]):
        test_hf['HouseFloor'][i]=test_hf[1][i]
    elif not np.isnan(test_hf[2][i]):
        test_hf['HouseFloor'][i]=test_hf[2][i]

test_hf=test_hf.drop([1,2],axis=1)
test['HouseFloor']=test_hf

test.loc[test['Ecology_2']=='B','Ecology_2']=1
test.loc[test['Ecology_2']=='A','Ecology_2']=0
test.loc[test['Ecology_3']=='A','Ecology_3']=1
test.loc[test['Ecology_3']=='B','Ecology_3']=0
test.loc[test['Shops_2']=='B','Shops_2']=1
test.loc[test['Shops_2']=='A','Shops_2']=0

<ipython-input-17-fc7384bf3e15>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kitch_sq[kitch_sq==1]=kitch_sq_med #заменяем медиану


Empty DataFrame
Columns: [DistrictId, Rooms, Square, LifeSquare, KitchenSquare, Floor, HouseFloor, HouseYear, Ecology_1, Ecology_2, Ecology_3, Social_1, Social_2, Social_3, Healthcare_1, Helthcare_2, Shops_1, Shops_2]
Index: []
Series([], Name: HouseYear, dtype: int64)
884 0 131
1015


<ipython-input-17-fc7384bf3e15>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hou_err1['HouseFloor']=hou_err1['Floor']
<ipython-input-17-fc7384bf3e15>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hou_err3['HouseFloor']=hou_err3['Floor']


In [18]:
#загружаем тестовые данные в модель и вычисляем результат
test_pred=reg.predict(test.to_numpy())
test_pred=pd.Series(test_pred)

test_pred_to_save=pd.read_csv(os.path.join(wrk,'sample_submission.csv')) #шаблон для загрузки на Kaggle
test_pred_to_save['Price']=test_pred
test_pred_to_save.to_csv(os.path.join(wrk,'test_pred.csv'),index=False)